In [1]:
import os
import pandas as pd
import re

In [2]:
target = "NepQuake15/RawNepQuake15/"
file_list = [os.path.join(target,x) for x in os.listdir(target)]
file_list.sort()
file_list

['NepQuake15/RawNepQuake15/Nepali-Tweets-04-25.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-26.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-27.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-28.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-29.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-30.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-01.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-02.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-03.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-04.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-11.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-12.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-13.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-14.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-15.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-16.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-17.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-18.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-19

In [3]:
#### CLEANING ##### remove @usernames and external-urls
def cleanText(pdf):
    pdf['Text'] = pdf.apply(lambda x: re.sub('@[a-zA-Z]+','',x['Text']),axis=1) # remove @usernames
    pdf['Text'] = pdf.apply(lambda x: re.sub('#[a-zA-Z]+','',x['Text']),axis=1) # remove hash-tag
    pdf['Text'] = pdf.apply(lambda x: re.sub('\w+:\/\/[\w.\/]+',' ',x['Text']),axis=1)  # remove external urls
    pdf['Text'] = pdf.apply(lambda x: re.sub('[a-zA-Z]+','',x['Text']),axis=1)  # remove english alphabets
    pdf['Text'] = pdf.apply(lambda x: re.sub("^\s[-_]\s*|\s*-\s*$",'',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\.+",'.',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub(",+",',',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\?+",'?',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("!+",'!',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\*+",'*',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("।+",'।',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub('"+','"',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\n+","\n",re.sub('( +\n +)+','\n',x['Text'])),axis=1) # remove new line feeds and extra spaces
    pdf['Text'] = pdf.apply(lambda x: re.sub(" +"," ",x['Text']).strip(),axis=1) # removing spaces
    return pdf


In [4]:
agg = pd.DataFrame()

In [5]:
for file in file_list:
    print(file)
    pdf = pd.read_csv(file)
    print('before',pdf.shape)
    pdf = cleanText(pdf)
    
    pdf.index = range(pdf.shape[0])
    
    ### Checking for shortest string in the dataframe
    lengths = pdf.apply(lambda x: len(x['Text'].split(" ")),axis=1)
    print("Min",min(lengths),"Max", max(lengths),"Avg",sum(lengths)/lengths.shape[0])
    
    ##### CLEANING ##### remove short sentences ----
    indexes = pdf.apply(lambda x: len(x['Text'].split(" "))>=5,axis=1)
    pdf = pdf[indexes]
    
    ### Drop Duplicates
    pdf = pdf.drop_duplicates()
    print("after",pdf.shape)
    
    if not agg.empty:
        agg = pd.concat([agg,pdf])
    else:
        agg = pdf
    
agg = agg.drop_duplicates()

NepQuake15/RawNepQuake15/Nepali-Tweets-04-25.csv
before (20500, 2)
Min 1 Max 35 Avg 10.984878048780487
after (16777, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-26.csv
before (10001, 2)
Min 1 Max 35 Avg 12.28997100289971
after (8274, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-27.csv
before (10001, 2)
Min 1 Max 36 Avg 12.22007799220078
after (8285, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-28.csv
before (6739, 2)
Min 1 Max 35 Avg 12.888410743433743
after (5242, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-29.csv
before (7313, 2)
Min 1 Max 30 Avg 12.645972924928211
after (6083, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-30.csv
before (10001, 2)
Min 1 Max 33 Avg 12.574342565743425
after (8587, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-01.csv
before (15001, 2)
Min 1 Max 35 Avg 12.04399706686221
after (12209, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-02.csv
before (15001, 2)
Min 1 Max 35 Avg 11.725684954336378
after (12378, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-03.csv
b

In [6]:
agg = agg.sort_values(by=['Date'])
agg['Date'].value_counts(), agg.shape

(Date
 2015-04-25    16777
 2015-05-18    13060
 2015-05-20    12999
 2015-05-13    12895
 2015-05-21    12894
 2015-05-14    12888
 2015-05-17    12877
 2015-05-15    12845
 2015-05-11    12821
 2015-05-16    12788
 2015-05-19    12785
 2015-05-12    12747
 2015-05-04    12542
 2015-05-02    12378
 2015-05-03    12322
 2015-05-01    12209
 2015-04-30     8587
 2015-04-27     8285
 2015-04-26     8274
 2015-04-29     6083
 2015-11-30     5817
 2015-04-28     5242
 2015-12-29     4141
 Name: count, dtype: int64,
 (254256, 2))

In [7]:
agg.to_csv("NepQuake15/finalized.csv",index=False)

In [8]:
test = agg[agg.apply(lambda x: len(x['Text'].split(" "))>=10,axis=1)]
test = test.sort_values(by=['Date'])
test['Date'].value_counts(), test.shape

(Date
 2015-04-25    10132
 2015-05-01     8201
 2015-05-02     8067
 2015-05-14     8021
 2015-05-12     7938
 2015-05-15     7907
 2015-05-13     7907
 2015-05-18     7848
 2015-05-03     7806
 2015-05-04     7788
 2015-05-20     7776
 2015-05-21     7749
 2015-05-16     7736
 2015-05-17     7724
 2015-05-11     7688
 2015-05-19     7469
 2015-04-30     5946
 2015-04-26     5627
 2015-04-27     5607
 2015-04-29     4222
 2015-04-28     3714
 2015-11-30     3555
 2015-12-29     2485
 Name: count, dtype: int64,
 (158913, 2))

In [9]:
test.to_csv("NepQuake15/finalized.csv",index=False)